<a href="https://colab.research.google.com/github/bonareri/Bitcoin-Prediction-Analysis/blob/main/data_collection_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1: Install Required Libraries

In [126]:
!pip install yfinance

## Importing Libraries

In [127]:
# Data handling and visualization
import yfinance as yf
import pandas as pd
# Date handling
from datetime import datetime

In [128]:
# Get Bitcoin historical data
btc = yf.Ticker("BTC-USD")  # Bitcoin in USD
btc_hist = btc.history(period="10y")  # Fetch last 10 years of data

In [129]:
# Display first 5 rows
print(btc_hist.head())

                                 Open        High         Low       Close  \
Date                                                                        
2015-02-06 00:00:00+00:00  216.923004  230.509995  216.231995  222.266006   
2015-02-07 00:00:00+00:00  222.632996  230.298996  222.606995  227.753998   
2015-02-08 00:00:00+00:00  227.692993  229.438004  221.076996  223.412003   
2015-02-09 00:00:00+00:00  223.389008  223.977005  217.018997  220.110001   
2015-02-10 00:00:00+00:00  220.281998  221.807007  215.332001  219.839005   

                             Volume  Dividends  Stock Splits  
Date                                                          
2015-02-06 00:00:00+00:00  24435300        0.0           0.0  
2015-02-07 00:00:00+00:00  21604200        0.0           0.0  
2015-02-08 00:00:00+00:00  17145200        0.0           0.0  
2015-02-09 00:00:00+00:00  27791300        0.0           0.0  
2015-02-10 00:00:00+00:00  21115100        0.0           0.0  


In [130]:
# Save to CSV
btc_hist.to_csv("bitcoin_prices_yfinance.csv")

In [131]:
# Read the CSV file with the Date column parsed as datetime and set as index
df = pd.read_csv("bitcoin_prices_yfinance.csv", index_col='Date', parse_dates=True)

# Display the first few rows to verify
print("Initial Data:")
print(df.head())

Initial Data:
                                 Open        High         Low       Close  \
Date                                                                        
2015-02-06 00:00:00+00:00  216.923004  230.509995  216.231995  222.266006   
2015-02-07 00:00:00+00:00  222.632996  230.298996  222.606995  227.753998   
2015-02-08 00:00:00+00:00  227.692993  229.438004  221.076996  223.412003   
2015-02-09 00:00:00+00:00  223.389008  223.977005  217.018997  220.110001   
2015-02-10 00:00:00+00:00  220.281998  221.807007  215.332001  219.839005   

                             Volume  Dividends  Stock Splits  
Date                                                          
2015-02-06 00:00:00+00:00  24435300        0.0           0.0  
2015-02-07 00:00:00+00:00  21604200        0.0           0.0  
2015-02-08 00:00:00+00:00  17145200        0.0           0.0  
2015-02-09 00:00:00+00:00  27791300        0.0           0.0  
2015-02-10 00:00:00+00:00  21115100        0.0           0.0  


## Data Cleaning

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3654 entries, 2015-02-06 00:00:00+00:00 to 2025-02-06 00:00:00+00:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          3654 non-null   float64
 1   High          3654 non-null   float64
 2   Low           3654 non-null   float64
 3   Close         3654 non-null   float64
 4   Volume        3654 non-null   int64  
 5   Dividends     3654 non-null   float64
 6   Stock Splits  3654 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 228.4 KB


In [133]:
# Drop the 'Dividends' and 'Stock Splits' columns
df.drop(columns=["Dividends", "Stock Splits"], inplace=True)

In [134]:
# Remove timezone info from the index, if present
df.index = df.index.tz_localize(None)

In [135]:
# Ensure the DataFrame is sorted by date (ascending order)
df.sort_index(inplace=True)

In [136]:
#Ensure date is set as the index
df.index = pd.to_datetime(df.index)

In [137]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2015-02-06,216.923004,230.509995,216.231995,222.266006,24435300
2015-02-07,222.632996,230.298996,222.606995,227.753998,21604200
2015-02-08,227.692993,229.438004,221.076996,223.412003,17145200
2015-02-09,223.389008,223.977005,217.018997,220.110001,27791300
2015-02-10,220.281998,221.807007,215.332001,219.839005,21115100


In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3654 entries, 2015-02-06 to 2025-02-06
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    3654 non-null   float64
 1   High    3654 non-null   float64
 2   Low     3654 non-null   float64
 3   Close   3654 non-null   float64
 4   Volume  3654 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 171.3 KB


## Feature Engineering

- Daily Returns: Percentage change between consecutive closing prices.
- Moving Averages: 7-day and 30-day moving averages to smooth price fluctuations.

In [139]:
# Calculate daily percentage return based on the 'Close' price
df["Daily_Return"] = df["Close"].pct_change()

# Calculate 7-day and 30-day moving averages of the 'Close' price
df["MA7"] = df["Close"].rolling(window=7).mean()
df["MA30"] = df["Close"].rolling(window=30).mean()

In [140]:
df.head()

,Open,High,Low,Close,Volume,Daily_Return,MA7,MA30
Date,,,,,,,,
2015-02-06,216.923004,230.509995,216.231995,222.266006,24435300,NaN,NaN,NaN
2015-02-07,222.632996,230.298996,222.606995,227.753998,21604200,0.024691,NaN,NaN
2015-02-08,227.692993,229.438004,221.076996,223.412003,17145200,-0.019064,NaN,NaN
2015-02-09,223.389008,223.977005,217.018997,220.110001,27791300,-0.014780,NaN,NaN
2015-02-10,220.281998,221.807007,215.332001,219.839005,21115100,-0.001231,NaN,NaN


In [141]:
df.isnull().sum()

,0
Open,0
High,0
Low,0
Close,0
Volume,0
Daily_Return,1
MA7,6
MA30,29


In [142]:
# Drop rows with missing values
df_clean = df.dropna()

In [143]:
#check misiing values
df_clean.isnull().sum()

,0
Open,0
High,0
Low,0
Close,0
Volume,0
Daily_Return,0
MA7,0
MA30,0


## Save Preprocessed Data

In [144]:
# Save the cleaned (preprocessed) data to a new CSV file
df_clean.to_csv("bitcoin_prices_preprocessed.csv")

print("Preprocessed data saved as 'bitcoin_prices_preprocessed.csv'")
print(df_clean.head())

Preprocessed data saved as 'bitcoin_prices_preprocessed.csv'
                  Open        High         Low       Close    Volume  \
Date                                                                   
2015-03-07  272.294006  277.854004  270.132996  276.260986  17825900   
2015-03-08  276.433014  277.858002  272.565002  274.354004  22067900   
2015-03-09  274.812012  292.700989  273.893005  289.606995  59178200   
2015-03-10  289.862000  300.044006  289.743011  291.760010  67770800   
2015-03-11  291.524994  297.390991  290.507996  296.378998  33963900   

            Daily_Return         MA7        MA30  
Date                                              
2015-03-07      0.012973  273.689715  244.523235  
2015-03-08     -0.006903  275.711430  246.259502  
2015-03-09      0.055596  277.702427  248.321268  
2015-03-10      0.007434  279.139287  250.599535  
2015-03-11      0.015831  282.465999  253.141835  
